In [1]:
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from keras.layers.convolutional import Conv2D
import random as rn
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import codecs
from tensorflow.contrib.rnn import *

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

In [4]:
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape

(10039, 200, 189, 1)

In [5]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss


In [6]:
# split train/test
split = 7869

In [7]:
# API model
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
#     #text network
#     input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
#     net_text = Embedding(nb_words,
#                     EMBEDDING_DIM,
#                     weights = [g_word_embedding_matrix],
#                     trainable = True)(input_text)
#     net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
#     net = Bidirectional(LSTM(64, return_sequences=False))(net)  
#     model_text = Dense(64)(net)
    
    #mocap network
    input_mocap = Input(shape=(200,189,1))
    model_mocap0 = BatchNormalization()(input_mocap)
    model_mocap1 = Conv2D(64, 3, strides=(3, 3), padding="same")(model_mocap0)
    model_mocap2 = Dropout(0.2)(model_mocap1)
    model_mocap3 = Activation('relu')(model_mocap2)
    model_mocap4 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap3)
    model_mocap5 = Dropout(0.2)(model_mocap4)
    model_mocap6 = Activation('relu')(model_mocap5)
    model_mocap7 = Conv2D(256, 3,strides=(3, 3), padding="same")(model_mocap6)
    model_mocap8 = Dropout(0.2)(model_mocap7) 
    model_mocap9 = Activation('relu')(model_mocap8)
    model_mocap10 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap9)   
    model_mocap11 = Dropout(0.2)(model_mocap10)  
    model_mocap12 = Activation('relu')(model_mocap11)
    model_mocap13 = Conv2D(64, 3,strides=(3, 3), padding="same")(model_mocap12)  
    model_mocap14 = Dropout(0.2)(model_mocap13) 
    model_mocap15 = Activation('relu')(model_mocap14)
    model_mocap = Flatten()(model_mocap15) 
    #model_mocap = Dropout(dropout_rate)(model_mocap16)
    # combined model
    model_combined = concatenate([model_speech, model_mocap])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_speech, input_mocap], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model_1 = model()
# model_1.summary()

In [8]:
# #speech_to_npy
# model1 = model()
# earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
#                               restore_best_weights=True)
# hist1 = model1.fit([feat[:split],x_train_mocap[:split]], vad[:split].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, shuffle=True,
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate([feat[split:],x_train_mocap[split:]], vad[split:].T.tolist())
# print(eval_metrik1)

# # make prediction
# predict = model1.predict([feat[6290:],x_train_mocap[6290:]], batch_size=32)
# np.save('speech_mocap_npy-batch7869', 
#          np.array(predict).reshape(3, 3749).T)


In [9]:
# main function for for LOSO
def main(time):
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:split],x_train_mocap[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[split:],x_train_mocap[split:]], vad[split:].T.tolist())
    predict = model_1.predict([feat[6290:],x_train_mocap[6290:]], batch_size=32)
    np.save('npy7869/speech_mocap_npy-7869batch'+str(time)+'.npy', 
             np.array(predict).reshape(3, 3749).T)
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('npy7869/sm7869-batch-1.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 54s 9ms/step - loss: 1.7675 - v_loss: 0.7114 - a_loss: 0.4849 - d_loss: 0.5698 - v_ccc: 0.2879 - a_ccc: 0.5148 - d_ccc: 0.4299 - val_loss: 2.4393 - val_v_loss: 0.8069 - val_a_loss: 0.8237 - val_d_loss: 0.8018 - val_v_ccc: 0.1896 - val_a_ccc: 0.1764 - val_d_ccc: 0.1947
Epoch 2/50
6295/6295 [==============================] - 45s 7ms/step - loss: 1.2506 - v_loss: 0.4382 - a_loss: 0.3520 - d_loss: 0.4613 - v_ccc: 0.5620 - a_ccc: 0.6484 - d_ccc: 0.5390 - val_loss: 1.9093 - val_v_loss: 0.7336 - val_a_loss: 0.5425 - val_d_loss: 0.6358 - val_v_ccc: 0.2646 - val_a_ccc: 0.4616 - val_d_ccc: 0.3645
Epoch 3/50
6295/6295 [==============================] - 45s 7ms/step - loss: 1.1840 - v_loss: 0.4031 - a_loss: 0.3378 - d_loss: 0.4426 - v_ccc: 0.5966 - a_ccc: 0.6620 - d_ccc: 0.5574 - val_loss: 1.7930 - val_v_loss: 0.76

Epoch 8/50
6295/6295 [==============================] - 40s 6ms/step - loss: 1.0702 - v_loss: 0.3546 - a_loss: 0.3126 - d_loss: 0.4026 - v_ccc: 0.6454 - a_ccc: 0.6873 - d_ccc: 0.5971 - val_loss: 1.8038 - val_v_loss: 0.7665 - val_a_loss: 0.4576 - val_d_loss: 0.5819 - val_v_ccc: 0.2321 - val_a_ccc: 0.5448 - val_d_ccc: 0.4193
Epoch 9/50
6295/6295 [==============================] - 41s 7ms/step - loss: 1.0383 - v_loss: 0.3424 - a_loss: 0.3031 - d_loss: 0.3924 - v_ccc: 0.6573 - a_ccc: 0.6968 - d_ccc: 0.6076 - val_loss: 1.7188 - val_v_loss: 0.6901 - val_a_loss: 0.4451 - val_d_loss: 0.5895 - val_v_ccc: 0.3112 - val_a_ccc: 0.5584 - val_d_ccc: 0.4116
Epoch 10/50
6295/6295 [==============================] - 41s 7ms/step - loss: 1.0376 - v_loss: 0.3384 - a_loss: 0.3031 - d_loss: 0.3958 - v_ccc: 0.6618 - a_ccc: 0.6967 - d_ccc: 0.6038 - val_loss: 1.8360 - val_v_loss: 0.7989 - val_a_loss: 0.4598 - val_d_loss: 0.5817 - val_v_ccc: 0.1989 - val_a_ccc: 0.5449 - val_d_ccc: 0.4201
Epoch 11/50
6295/6295 [=

6295/6295 [==============================] - 45s 7ms/step - loss: 1.0686 - v_loss: 0.3486 - a_loss: 0.3125 - d_loss: 0.4087 - v_ccc: 0.6518 - a_ccc: 0.6879 - d_ccc: 0.5917 - val_loss: 1.8079 - val_v_loss: 0.7540 - val_a_loss: 0.4548 - val_d_loss: 0.6022 - val_v_ccc: 0.2455 - val_a_ccc: 0.5471 - val_d_ccc: 0.3995
Epoch 8/50
6295/6295 [==============================] - 45s 7ms/step - loss: 1.0591 - v_loss: 0.3398 - a_loss: 0.3153 - d_loss: 0.4040 - v_ccc: 0.6602 - a_ccc: 0.6847 - d_ccc: 0.5960 - val_loss: 1.7870 - val_v_loss: 0.7582 - val_a_loss: 0.4503 - val_d_loss: 0.5830 - val_v_ccc: 0.2407 - val_a_ccc: 0.5536 - val_d_ccc: 0.4187
Epoch 9/50
6295/6295 [==============================] - 45s 7ms/step - loss: 1.0342 - v_loss: 0.3294 - a_loss: 0.3102 - d_loss: 0.3952 - v_ccc: 0.6710 - a_ccc: 0.6899 - d_ccc: 0.6049 - val_loss: 1.7888 - val_v_loss: 0.7710 - val_a_loss: 0.4702 - val_d_loss: 0.5534 - val_v_ccc: 0.2276 - val_a_ccc: 0.5348 - val_d_ccc: 0.4487
Epoch 10/50
6295/6295 [=============

Epoch 14/50
6295/6295 [==============================] - 44s 7ms/step - loss: 0.9689 - v_loss: 0.3107 - a_loss: 0.2880 - d_loss: 0.3700 - v_ccc: 0.6893 - a_ccc: 0.7119 - d_ccc: 0.6299 - val_loss: 1.7777 - val_v_loss: 0.7160 - val_a_loss: 0.4616 - val_d_loss: 0.5985 - val_v_ccc: 0.2817 - val_a_ccc: 0.5385 - val_d_ccc: 0.4022
Epoch 15/50
6295/6295 [==============================] - 44s 7ms/step - loss: 0.9681 - v_loss: 0.3091 - a_loss: 0.2917 - d_loss: 0.3669 - v_ccc: 0.6908 - a_ccc: 0.7082 - d_ccc: 0.6329 - val_loss: 1.8412 - val_v_loss: 0.7152 - val_a_loss: 0.4998 - val_d_loss: 0.6315 - val_v_ccc: 0.2845 - val_a_ccc: 0.5030 - val_d_ccc: 0.3713
Epoch 16/50
6295/6295 [==============================] - 44s 7ms/step - loss: 0.9559 - v_loss: 0.3086 - a_loss: 0.2835 - d_loss: 0.3639 - v_ccc: 0.6919 - a_ccc: 0.7163 - d_ccc: 0.6360 - val_loss: 1.8995 - val_v_loss: 0.7994 - val_a_loss: 0.4735 - val_d_loss: 0.6286 - val_v_ccc: 0.1980 - val_a_ccc: 0.5285 - val_d_ccc: 0.3740
Epoch 17/50
6295/6295 

Epoch 17/50
6295/6295 [==============================] - 44s 7ms/step - loss: 0.9453 - v_loss: 0.3028 - a_loss: 0.2861 - d_loss: 0.3561 - v_ccc: 0.6972 - a_ccc: 0.7139 - d_ccc: 0.6436 - val_loss: 1.7585 - val_v_loss: 0.6838 - val_a_loss: 0.4802 - val_d_loss: 0.5963 - val_v_ccc: 0.3146 - val_a_ccc: 0.5227 - val_d_ccc: 0.4042
Epoch 18/50
6295/6295 [==============================] - 44s 7ms/step - loss: 0.9461 - v_loss: 0.3014 - a_loss: 0.2870 - d_loss: 0.3572 - v_ccc: 0.6985 - a_ccc: 0.7128 - d_ccc: 0.6425 - val_loss: 1.8192 - val_v_loss: 0.7434 - val_a_loss: 0.4765 - val_d_loss: 0.6001 - val_v_ccc: 0.2533 - val_a_ccc: 0.5255 - val_d_ccc: 0.4019
Epoch 19/50
6295/6295 [==============================] - 45s 7ms/step - loss: 0.9394 - v_loss: 0.2949 - a_loss: 0.2870 - d_loss: 0.3574 - v_ccc: 0.7050 - a_ccc: 0.7129 - d_ccc: 0.6426 - val_loss: 1.7601 - val_v_loss: 0.7211 - val_a_loss: 0.4672 - val_d_loss: 0.5723 - val_v_ccc: 0.2757 - val_a_ccc: 0.5363 - val_d_ccc: 0.4279
Epoch 20/50
6295/6295 

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6295/6295 [==============================] - 41s 6ms/step - loss: 0.9967 - v_loss: 0.3198 - a_loss: 0.2957 - d_loss: 0.3819 - v_ccc: 0.6804 - a_ccc: 0.7046 - d_ccc: 0.6184 - val_loss: 1.7533 - val_v_loss: 0.7508 - val_a_loss: 0.4577 - val_d_loss: 0.5471 - val_v_ccc: 0.2498 - val_a_ccc: 0.5445 - val_d_ccc: 0.4524
Epoch 14/50
6295/6295 [==============================] - 42s 7ms/step - loss: 0.9789 - v_loss: 0.3168 - a_loss: 0.2910 - d_loss: 0.3706 - v_ccc: 0.6830 - a_ccc: 0.7089 - d_ccc: 0.6292 - val_loss: 1.7838 - val_v_loss: 0.7376 - val_a_loss: 0.4633 - val_d_loss: 0.5846 - val_v_ccc: 0.2615 - val_a_ccc: 0.5377 - val_d_ccc: 0.4170
Epoch 15/50
6295/6295 [==============================] - 41s 7ms/step - loss: 0.9656 - v_loss: 0.3101 - a_loss: 0.2902 - d_loss: 0.3655 - v_ccc: 0.6898 - a_ccc: 0.7099 - d_ccc: 0.6346 - val_loss: 1.7998 - val_v_loss: 0.7568 - val_a_loss: 0.4678 - val_d_loss: 0.5800 - val_v_ccc: 0.2450 - val_a_ccc: 0.5343 - val_d_ccc: 0.4209
Epoch 16/50
6295/6295 [===========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6295/6295 [==============================] - 47s 8ms/step - loss: 1.0909 - v_loss: 0.3535 - a_loss: 0.3247 - d_loss: 0.4129 - v_ccc: 0.6465 - a_ccc: 0.6753 - d_ccc: 0.5873 - val_loss: 1.7544 - val_v_loss: 0.7088 - val_a_loss: 0.4716 - val_d_loss: 0.5829 - val_v_ccc: 0.2936 - val_a_ccc: 0.5309 - val_d_ccc: 0.4212
Epoch 7/50
6295/6295 [==============================] - 45s 7ms/step - loss: 1.0600 - v_loss: 0.3401 - a_loss: 0.3153 - d_loss: 0.4046 - v_ccc: 0.6600 - a_ccc: 0.6847 - d_ccc: 0.5954 - val_loss: 1.7755 - val_v_loss: 0.7377 - val_a_loss: 0.4675 - val_d_loss: 0.5754 - val_v_ccc: 0.2627 - val_a_ccc: 0.5345 - val_d_ccc: 0.4272
Epoch 8/50
6295/6295 [==============================] - 44s 7ms/step - loss: 1.0492 - v_loss: 0.3418 - a_loss: 0.3114 - d_loss: 0.3956 - v_ccc: 0.6581 - a_ccc: 0.6884 - d_ccc: 0.6043 - val_loss: 1.7757 - val_v_loss: 0.7469 - val_a_loss: 0.4550 - val_d_loss: 0.5791 - val_v_ccc: 0.2538 - val_a_ccc: 0.5473 - val_d_ccc: 0.4232
Epoch 9/50
6295/6295 [==============

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

